# Use [TeleGram-Scraper](https://github.com/th3unkn0n/TeleGram-Scraper) to export users from a group chat
> small adjustment required if you get Flood errors: remove _aggressive=True_ in line 80 of scraper.py [details](https://github.com/LonamiWebs/Telethon/issues/1299#issuecomment-543246208)

# Good [start point](https://towardsdatascience.com/a-simple-exploration-of-a-telegram-chat-group-540a1e96ad75) for looking into messages stats 

---

## Tables
### members_df - list of group members exported with TeleGram-Scraper
    Data columns (total 6 columns):
    #   Column       Non-Null Count  Dtype 
    ---  ------       --------------  ----- 
    0   username     408 non-null    object
    1   user_id      483 non-null    object
    2   access_hash  483 non-null    object
    3   name         482 non-null    object
    4   group        483 non-null    object
    5   group_id     483 non-null    object
### msg_df - table with all messages including service messages
### msg_df_messages - only messages from everyone
    Data columns (total 3 columns):
    #   Column  Non-Null Count  Dtype 
    ---  ------  --------------  ----- 
    0   date    45174 non-null  object
    1   text    45174 non-null  object
    2   from    45162 non-null  object

### msg_df_joined - service messages about joined or added members
    Data columns (total 6 columns):
    #   Column         Non-Null Count  Dtype 
    ---  ------         --------------  ----- 
    0   date           797 non-null    object
    1   action         797 non-null    object
    2   actor          777 non-null    object
    3   added_members  797 non-null    object
    4   joiners_count  797 non-null    int64 
    5   join_type      797 non-null    object
    
### msg_df_removed - service messages about removed members (no bans)
    Data columns (total 5 columns):
    #   Column           Non-Null Count  Dtype 
    ---  ------           --------------  ----- 
    0   date             3 non-null      object
    1   action           3 non-null      object
    2   actor            3 non-null      object
    3   deleted_members  3 non-null      object
    4   removed_count    3 non-null      int64 

### spies - users who wrote zero messages

### quitters - users who is not a member of export

### current_usr_msgs - messages from actual users (skip quitters)

### out_df - df with usernames and number of messages ready for export



In [ ]:
import pandas as pd
import json
import plotly.express as px

chat_export = r'.\result.json'
members = r'.\members.csv'
#after_date = '2022-01-01' # uncomment if you want to work with data starting from a particular date

#open json file
with open(chat_export, encoding='utf8') as f:
    d = json.load(f)

members_df = pd.read_csv(members, dtype=str) #read members.csv
members_df.columns = members_df.columns.str.replace(' ','_') #replace spaces in column names with underscore

#normalize it according to parent node
norm_msg = pd.json_normalize(d['messages'])
msg_df = pd.DataFrame(norm_msg) # put messages into a dataframe
msg_df.from_id = msg_df.from_id.str.replace('user','')
if 'after_date' in globals(): # if after_date defined in global context then filter-out everything before that date
    msg_df = msg_df[(msg_df['date'] >= after_date)]

# show messages dataframe
#msg_df.head()

# show all message types
#print(msg_df['type'].unique())

# show all action types
#print(msg_df['action'].unique())

In [ ]:
msg_df_messages = msg_df[msg_df.type=='message'] #filter only message type message 
msg_df_messages = msg_df_messages[['date','text','from','from_id']] #filter important columns 

msg_df_joined = msg_df[(msg_df.action=='invite_members') | (msg_df.action=='join_group_by_link')] #filter only invite_members actions
msg_df_joined = msg_df_joined[['date','action','actor','members']] #filter important columns 
msg_df_joined = msg_df_joined.rename({'members': 'added_members'}, axis='columns')
msg_df_joined['joiners_count']=msg_df_joined.added_members.str.len() #add number of joined accounts per action
msg_df_joined['join_type'] = msg_df_joined[msg_df_joined.action!='join_group_by_link'].apply\
    (
    lambda row:\
        'joined' if (row.actor in row.added_members) and row.joiners_count == 1\
        else 'added', axis=1\
    ) #add join type: if actor in added_members and count of added_members is 1 then self-join otherwise added by actor


msg_df_removed = msg_df[msg_df.action=='remove_members'] #filter only invite_members actions
msg_df_removed = msg_df_removed[['date','action','actor','members']] #filter important columns
msg_df_removed = msg_df_removed.rename({'members': 'deleted_members'}, axis='columns')
msg_df_removed['removed_count']=msg_df_removed.deleted_members.str.len() #add number of removed accounts per action


# msg_df_joined.info()
# msg_df_removed.info()
# msg_df_messages.info()


In [ ]:
merged = pd.merge(msg_df_messages, members_df, left_on='from_id', right_on='user_id', how='outer') # full outer join of members list with messages df
spies = merged[merged['from_id'].isnull()].copy()
#spies_compact = spies_full[['username', 'user_id', 'name']] # get only users from members df who wrote zero messages

quitters = merged[merged['user_id'].isnull()].copy() # get only users who has no entry in members export
#quitters_compact = quitters_full[['from', 'from_id', 'text', 'date']]

current_usr_msgs = merged.dropna(subset=['user_id']) # messages from actual users (skip quitters)

# spies.info()
# quitters.info()
# current_usr_msgs.info()

In [ ]:
# print some numbers
print(f'{len(msg_df_messages.from_id.unique())} - # of unique users who wrote something')
print(f'{len(members_df.user_id.unique())} - # of group members in exported list')
print(f'{len(spies.user_id.unique())} - # of spies (wrote 0 messages)')
print(f'{len(quitters.from_id.unique())} - # of quitters (wrote something but not in members exported)')

In [ ]:
messages_sent_imp = current_usr_msgs['from_id'].value_counts()
messages_sent_imp = messages_sent_imp.reset_index().rename(columns={'index':'user_id', 'from_id':'Count'})

spies['Count'] = 0 # add column Count to spies df
messages_sent_imp = pd.concat([messages_sent_imp, spies[['user_id', 'Count']]], ignore_index=True) # add spies to messages_sent_imp df

fig_count_msgs = px.bar(messages_sent_imp.nlargest(20, 'Count'), x='user_id', y='Count', labels={'Count':'# of messages'})
fig_count_msgs.show()

In [ ]:
fig_hist_step1 = px.histogram(messages_sent_imp, x='Count', nbins=max(messages_sent_imp.Count)+1, range_x = list({0,15}), text_auto  = True)
#dict(r=[1, 2, range(3,10), range(11, 50), range(51, 100), range(101, 500), range(501, max(messages_sent_imp.Count))])
#messages_sent_imp.hist(bins=len(messages_sent_imp.Count.unique()))
fig_hist_step1.update_xaxes(title='# of messages sent')
fig_hist_step1.update_yaxes(title='# of users')
fig_hist_step1.show()


In [ ]:
out_df = pd.merge(messages_sent_imp, members_df, left_on='user_id', right_on='user_id') # df with usernames and number of messages ready for export